# Obtención de datos mediante API

## Weather API

Es un proveedor de API de geolocalización y clima totalmente administrado que porporciona API extensas que van desde pronosticos meteorológicos y en tiempo real, clima historico, datos de calidad del aire, irradancia solar, solicitud masiva, busqueda de IP y astronomía.
[Weather API](https://www.weatherapi.com/)
- Clima en tiempo real
- Previsión meteorologica por horas hasta 14 días
- Clima futuro
- Intervalos diarios y horarios
- Datos de calidad del aire
- Irradiancia sola
- Datos de ubicación

Esta practica tiene como objetivo obtener datos que corresponden al clima de los 570 municipios del estado de Oaxaca.
Esta API es capaz de proporcionarnos toda esta ínformación de manera gratuita.

In [2]:
import os
import numpy as np
import pandas as pd
import requests
import time
from dotenv import load_dotenv
from selenium import webdriver
from selenium.webdriver.common.by import By
from unidecode import unidecode

## Scraping para obtener nombre de los municipios

Para asegurarnos que tenemos todos los nombres y que sean correctos, vamos a hacer una recolección de estos a través de una página del inegi que los lista.

Vamos a utilizar la herramienta Selenium para abrir un navegador, ir al enlace del inegi y guardar los nombres en una lista.

[Documentación completa de Selenium](https://pypi.org/project/selenium/)

In [3]:
# Set up the WebDriver
driver = webdriver.Chrome()

# Open the website
url = "https://cuentame.inegi.org.mx/monografias/informacion/oax/territorio/div_municipal.aspx?tema=me&e=20"
driver.get(url)

# Wait to url is ready
time.sleep(5)

# Read and save information
oax_municipalities = []
for i in range(1, 571):
    state = driver.find_element(
        By.XPATH, "//*[@id='keywords2']/tbody/tr[" + str(i) + "]/td[2]"
    )
    # Save and clean string
    oax_municipalities.append(unidecode(state.text))

# Close the browser
driver.quit()

El tiempo que toma obtener los datos de la API se vuelve considerablemente grande con el número de municipios que nosotros deseemamos consultar, para hacer pruebas rapidas la opción de `debug` es util pues solo trabajamos con 100 elementos para revisar que todo salga bien. Cuando el código sea funcional ahora si, ya podemos solicitar la información completa.

In [4]:
debug = True
if debug:
    municipalities = oax_municipalities[0:100]
else:
    municipalities = oax_municipalities

## Envío de solicitudes a Weather API

Los datos que vamos a recolectar, nos lo van a proporcionar 3 request:
- search
- current
- astronomy

Se realizó una función que recibe un request y nos regresa una lista de objetos json con la información que nos proporciona la API.

Cada request llevará un nombre de municipio diferente, así que recorreremos la lista de los municipios y agregaremos el parametro `region:oaxaca` para filtrar la información y así no datos erroneos o de un lugar con el nombre parecido.

In [5]:
def getApiData(endpoint, payload, municipalities, getFirst):
    data_collected = []
    for municipality in municipalities:
        payload["q"] = "'name':'" + municipality + "','region':'oaxaca'"
        try:
            time.sleep(5)
            r = requests.get(url=endpoint, params=payload)
            if r.status_code == 200:
                json = r.json()
                if getFirst:
                    data_collected.append(json[0])
                else:
                    data_collected.append(json)
        except:
            continue
    return data_collected

Cargamos los datos de las variables de entorno del sistema

In [6]:
load_dotenv()
API_KEY = os.getenv("API_KEY")

In [7]:
endpoint = "http://api.weatherapi.com/v1/"
payload = {"key": API_KEY}

In [8]:
search = "/search.json"

In [9]:
data_search = getApiData(
    endpoint=endpoint + search,
    payload=payload,
    municipalities=municipalities,
    getFirst=True,
)

In [10]:
current = "/current.json"

In [9]:
data_current = getApiData(
    endpoint=endpoint + current,
    payload=payload,
    municipalities=municipalities,
    getFirst=False,
)

In [10]:
astronomy = "/astronomy.json"

In [11]:
data_astronomy = getApiData(
    endpoint=endpoint + astronomy,
    payload=payload,
    municipalities=municipalities,
    getFirst=False,
)

## Normalización de datos Json y Conversion a DataFrames

Una vez que recolectamos los datos, los llevamos a dataframes

In [12]:

normalized_data_search = pd.json_normalize(data_search)
df_search_temp = pd.DataFrame.from_dict(normalized_data_search).add_prefix("location.")
df_search = df_search_temp.drop(["location.lat", "location.lon"], axis=1)
df_search.head()

,location.id,location.name,location.region,location.country,location.url
0,3209265,Abejones,Oaxaca,Mexico,abejones-oaxaca-mexico
1,3217399,Acatlan De Perez Figueroa,Oaxaca,Mexico,acatlan-de-perez-figueroa-oaxaca-mexico
2,3209199,Asuncion Cacalotepec,Oaxaca,Mexico,asuncion-cacalotepec-oaxaca-mexico
3,3209198,Asuncion Cuyotepeji,Oaxaca,Mexico,asuncion-cuyotepeji-oaxaca-mexico
4,3217411,Asuncion Ixtaltepec,Oaxaca,Mexico,asuncion-ixtaltepec-oaxaca-mexico


In [13]:
normalized_data_current = pd.json_normalize(data_current)
df_current = pd.DataFrame.from_dict(normalized_data_current)
df_current.head()

,location.name,location.region,location.country,location.lat,location.lon,location.tz_id,location.localtime_epoch,location.localtime,current.last_updated_epoch,current.last_updated,...,current.windchill_f,current.heatindex_c,current.heatindex_f,current.dewpoint_c,current.dewpoint_f,current.vis_km,current.vis_miles,current.uv,current.gust_mph,current.gust_kph
0,Abejones,Oaxaca,Mexico,17.4364,-96.6085,America/Mexico_City,1735773399,2025-01-01 17:16,1735773300,2025-01-01 17:15,...,62.4,16.9,62.4,11.6,52.9,10.0,6.0,0.4,7.5,12.0
1,Acatlan De Perez Figueroa,Oaxaca,Mexico,18.5397,-96.6057,America/Mexico_City,1735771180,2025-01-01 16:39,1735770600,2025-01-01 16:30,...,80.0,29.2,84.5,21.8,71.2,10.0,6.0,1.4,11.3,18.3
2,Asuncion Cacalotepec,Oaxaca,Mexico,17.0361,-95.9550,America/Mexico_City,1735773372,2025-01-01 17:16,1735773300,2025-01-01 17:15,...,62.8,17.1,62.8,6.1,43.0,10.0,6.0,0.4,11.5,18.6
3,Asuncion Cuyotepeji,Oaxaca,Mexico,17.9167,-97.8000,America/Mexico_City,1735773437,2025-01-01 17:17,1735773300,2025-01-01 17:15,...,72.9,24.0,75.2,6.9,44.4,10.0,6.0,0.5,5.4,8.7
4,Asuncion Ixtaltepec,Oaxaca,Mexico,16.5031,-95.0608,America/Mexico_City,1735772630,2025-01-01 17:03,1735772400,2025-01-01 17:00,...,81.1,28.4,83.2,17.6,63.6,10.0,6.0,0.4,20.6,33.2


In [14]:
normalized_data_astronomy = pd.json_normalize(data_astronomy)
df_astronomy = pd.DataFrame.from_dict(normalized_data_astronomy)
df_astronomy.head()

,location.name,location.region,location.country,location.lat,location.lon,location.tz_id,location.localtime_epoch,location.localtime,astronomy.astro.sunrise,astronomy.astro.sunset,astronomy.astro.moonrise,astronomy.astro.moonset,astronomy.astro.moon_phase,astronomy.astro.moon_illumination,astronomy.astro.is_moon_up,astronomy.astro.is_sun_up
0,Abejones,Oaxaca,Mexico,17.4364,-96.6085,America/Mexico_City,1735777159,2025-01-01 18:19,06:57 AM,06:04 PM,09:22 AM,08:57 PM,Waxing Crescent,6,0,0
1,Acatlan De Perez Figueroa,Oaxaca,Mexico,18.5397,-96.6057,America/Mexico_City,1735777192,2025-01-01 18:19,06:59 AM,06:02 PM,09:23 AM,08:56 PM,Waxing Crescent,6,0,0
2,Asuncion Cacalotepec,Oaxaca,Mexico,17.0361,-95.9550,America/Mexico_City,1735774980,2025-01-01 17:43,06:53 AM,06:02 PM,08:31 AM,07:55 PM,Waxing Crescent,2,0,1
3,Asuncion Cuyotepeji,Oaxaca,Mexico,17.9167,-97.8000,America/Mexico_City,1735777213,2025-01-01 18:20,07:03 AM,06:08 PM,09:27 AM,09:01 PM,Waxing Crescent,6,0,0
4,Asuncion Ixtaltepec,Oaxaca,Mexico,16.5031,-95.0608,America/Mexico_City,1735777218,2025-01-01 18:20,06:49 AM,06:00 PM,09:14 AM,08:52 PM,Waxing Crescent,6,0,0


## Unión de información

Como pudimos ver, los dataframes contienen diferentes columnas para cada municipio. 
Es necesario unirlas para tener todas las columnas en una sola tabla, debido a que hay columnas en común, utilizaremos la herramienta
merge para mezclar las tablas tomando como referencia el `name`, `region`, `country`, `lat`, `lon`, `tz_id` que son columnas que tienen en común y así se emparejan los datos.

In [15]:
df_astronomy_and_current = pd.merge(
    df_astronomy,
    df_current,
    on=[
        "location.name",
        "location.region",
        "location.country",
        "location.lat",
        "location.lon",
        "location.tz_id",
    ],
)
df_astronomy_and_current.head()

,location.name,location.region,location.country,location.lat,location.lon,location.tz_id,location.localtime_epoch_x,location.localtime_x,astronomy.astro.sunrise,astronomy.astro.sunset,...,current.windchill_f,current.heatindex_c,current.heatindex_f,current.dewpoint_c,current.dewpoint_f,current.vis_km,current.vis_miles,current.uv,current.gust_mph,current.gust_kph
0,Abejones,Oaxaca,Mexico,17.4364,-96.6085,America/Mexico_City,1735777159,2025-01-01 18:19,06:57 AM,06:04 PM,...,62.4,16.9,62.4,11.6,52.9,10.0,6.0,0.4,7.5,12.0
1,Acatlan De Perez Figueroa,Oaxaca,Mexico,18.5397,-96.6057,America/Mexico_City,1735777192,2025-01-01 18:19,06:59 AM,06:02 PM,...,80.0,29.2,84.5,21.8,71.2,10.0,6.0,1.4,11.3,18.3
2,Asuncion Cacalotepec,Oaxaca,Mexico,17.0361,-95.9550,America/Mexico_City,1735774980,2025-01-01 17:43,06:53 AM,06:02 PM,...,62.8,17.1,62.8,6.1,43.0,10.0,6.0,0.4,11.5,18.6
3,Asuncion Cuyotepeji,Oaxaca,Mexico,17.9167,-97.8000,America/Mexico_City,1735777213,2025-01-01 18:20,07:03 AM,06:08 PM,...,72.9,24.0,75.2,6.9,44.4,10.0,6.0,0.5,5.4,8.7
4,Asuncion Ixtaltepec,Oaxaca,Mexico,16.5031,-95.0608,America/Mexico_City,1735777218,2025-01-01 18:20,06:49 AM,06:00 PM,...,81.1,28.4,83.2,17.6,63.6,10.0,6.0,0.4,20.6,33.2


El conjunto de datos `search` contiene un elemento muy importante que es el `id` de cada municipio, entonces vamos a mezclar la tabla que ya fue unida anteriormente con los datos de search tomando como referencias las columnas con el mismo nombre.

In [16]:
df_full = pd.merge(
    df_search,
    df_astronomy_and_current,
    on=[
        "location.name",
        "location.region",
        "location.country",
    ],
)

In [17]:
df_full.head()

,location.id,location.name,location.region,location.country,location.url,location.lat,location.lon,location.tz_id,location.localtime_epoch_x,location.localtime_x,...,current.windchill_f,current.heatindex_c,current.heatindex_f,current.dewpoint_c,current.dewpoint_f,current.vis_km,current.vis_miles,current.uv,current.gust_mph,current.gust_kph
0,3209265,Abejones,Oaxaca,Mexico,abejones-oaxaca-mexico,17.4364,-96.6085,America/Mexico_City,1735777159,2025-01-01 18:19,...,62.4,16.9,62.4,11.6,52.9,10.0,6.0,0.4,7.5,12.0
1,3217399,Acatlan De Perez Figueroa,Oaxaca,Mexico,acatlan-de-perez-figueroa-oaxaca-mexico,18.5397,-96.6057,America/Mexico_City,1735777192,2025-01-01 18:19,...,80.0,29.2,84.5,21.8,71.2,10.0,6.0,1.4,11.3,18.3
2,3209199,Asuncion Cacalotepec,Oaxaca,Mexico,asuncion-cacalotepec-oaxaca-mexico,17.0361,-95.9550,America/Mexico_City,1735774980,2025-01-01 17:43,...,62.8,17.1,62.8,6.1,43.0,10.0,6.0,0.4,11.5,18.6
3,3209198,Asuncion Cuyotepeji,Oaxaca,Mexico,asuncion-cuyotepeji-oaxaca-mexico,17.9167,-97.8000,America/Mexico_City,1735777213,2025-01-01 18:20,...,72.9,24.0,75.2,6.9,44.4,10.0,6.0,0.5,5.4,8.7
4,3217411,Asuncion Ixtaltepec,Oaxaca,Mexico,asuncion-ixtaltepec-oaxaca-mexico,16.5031,-95.0608,America/Mexico_City,1735777218,2025-01-01 18:20,...,81.1,28.4,83.2,17.6,63.6,10.0,6.0,0.4,20.6,33.2


## Exportación de datos

Una vez que los datos están en una sola tabla, lo guardaremos con el formato csv

In [18]:
df_full.to_csv("dataset/dataset_raw/weather.csv")